In [ ]:
import networkx as nx
import nxviz as nv
import pandas as pd
import matplotlib as plt
import community

In [ ]:
df_alt_influence = pd.read_csv('AltNews Report Check.csv')
df_alt_small = df_alt_influence.loc[df_alt_influence.Campaign == 'Influence'][['News_Number', 'Fact_Check_Link']]

In [ ]:
df_alt_influence

In [ ]:
df_alt_small = df_alt_influence.loc[df_alt_influence.Campaign == 'Influence'][['News_Number', 'Fact_Check_Link']]

In [ ]:
df_alt_all = pd.read_csv('Collected_Fake_Tweets_Still_Available.csv')
df_account = pd.read_csv('All_Fake_without_debunkers_Updated.csv')

In [ ]:
df_alt_all_influence = pd.merge(df_alt_all, df_alt_small, on = 'Fact_Check_Link').drop(columns = 'Unnamed: 0')
df_merge_info = pd.merge(df_alt_all_influence[['User_ID', 'News_Number']], df_account[['User_ID', 'Political', 'party', 'ScreenName']])

In [ ]:
len(set(df_merge_info.User_ID.tolist()))

In [ ]:
len(set(df_account.User_ID.tolist()))

In [ ]:
len(set(df_alt_all_influence.User_ID.tolist()))

In [ ]:
df_merge_info.sort_values('News_Number')

In [ ]:
df_merge_info.columns

In [ ]:
news_groups = df_merge_info.groupby('News_Number')['User_ID'].apply(set)
G = nx.Graph()

for news, users in news_groups.items():
    for user1 in users:
        for user2 in users:
            if user1 != user2:
                if not G.has_edge(user1, user2):
                    G.add_edge(user1, user2, weight=1)
                else:
                    G[user1][user2]['weight'] += 1

nx.draw(G, with_labels=True, node_color='skyblue', node_size=500, font_size=12)
plt.title('User Interaction Graph based on News Numbers')
plt.show()

In [ ]:
connected_components = list(nx.connected_components(G))

for i, component in enumerate(connected_components, 1):
    print(f"Connected Component {i}: {len(component)}")
    

In [ ]:
partition = community.best_partition(G)

communities = {}
for node, community_id in partition.items():
    if community_id not in communities:
        communities[community_id] = []
    communities[community_id].append(node)

for community_id, nodes in communities.items():
    print(f"Community {community_id}: {len(nodes)}")

In [ ]:
num_communities = len(set(partition.values()))


In [ ]:
sorted_communities = sorted(communities.items(), key=lambda x: len(x[1]), reverse=True)

for idx, (community_id, nodes) in enumerate(sorted_communities, 1):
    print(f"Community {idx} (ID: {community_id}), Number of Nodes: {len(nodes)}")

In [ ]:
partition = community.best_partition(G)

communities = {}
for node, community_id in partition.items():
    if community_id not in communities:
        communities[community_id] = []
    communities[community_id].append(node)



In [ ]:
partition = community.best_partition(G)

communities = {}
for node, community_id in partition.items():
    if community_id not in communities:
        communities[community_id] = []
    communities[community_id].append(node)

community_data = []

for community_id, nodes in communities.items():
    num_edges = G.subgraph(nodes).number_of_edges()
    max_edges = len(nodes) * (len(nodes) - 1) / 2
    density = num_edges / max_edges
    community_data.append((community_id, len(nodes), density))

df_acc_edge = pd.DataFrame(community_data, columns=['Community_ID', 'Num_Nodes', 'Density'])

print(df_acc_edge)

In [ ]:
nx.write_graphml(G, "AltNews_Account.graphml")

In [ ]:
nx.write_gpickle(G, "AltNews_Account.pkl")

In [ ]:
type(partition)

In [ ]:
import pickle


with open("Account_Partition.pkl", "wb") as pkl_file:
    pickle.dump(partition, pkl_file)

In [ ]:
modularity = community.modularity(partition, G)

In [ ]:
modularity

In [ ]:
for community_id, nodes in communities.items():
    subgraph = G.subgraph(nodes)
    modularity = community.modularity(partition, subgraph)
    
    print(f"Community {community_id}: Modularity = {modularity}")

In [ ]:
df_acc_edge = df_acc_edge.sort_values(by = 'Num_Nodes', ascending = False).reset_index(drop = True)

In [ ]:
df_acc_edge.loc[df_acc_edge.Num_Nodes >= 7]

In [ ]:
community.modularity(partition, G)

In [ ]:
df_acc_edge

In [ ]:
pd.DataFrame(communities)

In [ ]:
community_data = [(node, community_id) for node, community_id in partition.items()]

df_communities = pd.DataFrame(community_data, columns=['Node', 'Community_ID'])

In [ ]:
df_communities.sort_values('Community_ID').rename(columns = {'Node': 'User_ID'}).to_csv('AltNews_User_Community.csv', index = False)

In [ ]:
df_acc_edge.to_csv('AltNews_Community_Table.csv', index = False)

In [ ]:
df_communities.groupby('Community_ID').count()

In [ ]:
df_acc_edge = df_acc_edge.rename(columns = {'Num_Nodes': 'Number_of_Accounts'})

In [ ]:
df_acc_edge.loc[df_acc_edge.Density > 0.8].iloc[0:20]

In [ ]:
df_acc_edge.to_csv('AltNews_Community_Table.csv', index = False)

In [ ]:
df_acc_edge2 = pd.read_csv('AltNews_Community_Table.csv')

In [ ]:
df_acc_edge2

In [ ]:
len(df_acc_edge2)

In [ ]:
df_acc_edge2.loc[df_acc_edge2.Number_of_Accounts >= 5].mean()

In [ ]:
nodes_9 = communities[9]

In [ ]:
subgraph = G.subgraph(nodes_9)

In [ ]:
subgraph

In [ ]:
total_weight = sum(weight for _, _, weight in G.edges(data='weight'))
num_edges = G.number_of_edges()

average_weight = total_weight / num_edges

print("Average edge weight:", average_weight)

In [ ]:
nx.draw(subgraph, node_size=100)

In [ ]:
sub_edge = subgraph.number_of_edges()
sub_nodes = subgraph.number_of_nodes()

In [ ]:
sub_edge/((sub_nodes* (sub_nodes-1))/2)

In [ ]:
import networkx as nx
import community


partition3 = community.best_partition(G)

communities3 = {}
for node, community_id in partition3.items():
    if community_id not in communities3:
        communities3[community_id] = []
    communities3[community_id].append(node)

for community_id, nodes in communities3.items():
    subgraph3 = G.subgraph(nodes)
    modularity3 = community.modularity(partition3, subgraph3, weight='weight')
    print(f"Community {community_id}: Modularity = {modularity3}")


In [ ]:
nodes

In [ ]:
partition2 = community.best_partition(G)

for community_id in set(partition2.values()):
    nodes_in_community = [node for node, comm_id in partition.items() if comm_id == community_id]
    
    subgraph = G.subgraph(nodes_in_community)
    
    modularity = community.modularity(partition2, subgraph)
    
    print(f"Community {community_id}: Modularity = {modularity}")

In [ ]:
modularity

In [ ]:
partition2

In [ ]:
subgraph

In [ ]:
subgraph2 = G.subgraph(communities[9])

In [ ]:
num_edges =subgraph2.number_of_edges()

In [ ]:
num_edges

In [ ]:
total_weight = 0

for u, v, data in subgraph.edges(data=True):
    if 'weight' in data:
        total_weight += data['weight']

print("Total weight in the subgraph:", total_weight)

In [ ]:
nx.draw(subgraph, node_size=100)

In [ ]:
edge_weights = {}
for u, v, data in subgraph.edges(data=True):
    edge_weights[(u, v)] = data['weight']

for edge, weight in edge_weights.items():
    print(f"Edge {edge}: Weight {weight}")

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt



pos = nx.spring_layout(subgraph)  # You can use any layout you prefer
nx.draw(subgraph, pos, with_labels=True, node_color='skyblue', node_size=100, edge_color='black', linewidths=1, font_size=15)

edge_labels = nx.get_edge_attributes(subgraph, 'weight')
nx.draw_networkx_edge_labels(subgraph, pos, edge_labels=edge_labels, font_color='red')

plt.show()

In [ ]:
G2 = nx.Graph()

news_groups2 = df_merge_info.groupby('News_Number')['User_ID'].apply(set)

for news, users in news_groups2.items():
    for user1 in users:
        for user2 in users:
            if user1 < user2:  # Ensure unique pairs (user1, user2)
                if G2.has_edge(user1, user2):
                    G2[user1][user2]['weight'] += 1
                else:
                    G2.add_edge(user1, user2, weight=1)

nx.draw(G2, with_labels=True, node_color='skyblue', node_size=100, font_size=12)
plt.title('User Interaction Graph based on News Numbers')
plt.show()

In [ ]:
edge_weights = {}
for u, v, data in G2.edges(data=True):
    edge_weights[(u, v)] = data['weight']

for edge, weight in edge_weights.items():
    print(f"Edge {edge}: Weight {weight}")

In [ ]:
partition2 = community.best_partition(G2)

community_data2 = []

for community_id in set(partition2.values()):
    nodes_in_community = [node for node, comm_id in partition2.items() if comm_id == community_id]
    
    subgraph = G2.subgraph(nodes_in_community)
    
    num_nodes = subgraph.number_of_nodes()
    num_edges = subgraph.number_of_edges()
    density = nx.density(subgraph)
    avg_degree = sum(dict(subgraph.degree()).values()) / num_nodes
    clustering_coefficient = nx.average_clustering(subgraph)
    modularity = community.modularity(partition2, subgraph)
    
    edge_weights = [data['weight'] for _, _, data in subgraph.edges(data=True)]
    average_weight = sum(edge_weights) / len(edge_weights) if edge_weights else 0
    
    community_data2.append((community_id, num_nodes, num_edges, density, avg_degree, clustering_coefficient, average_weight))

df_community2 = pd.DataFrame(community_data2, columns=['Community_ID', 'Num_Nodes', 'Num_Edges', 'Density', 'Avg_Degree', 'Clustering_Coefficient', 'average_weight'])


In [ ]:
df_community2.sort_values('Num_Nodes', ascending = False).reset_index(drop = True).to_csv('AltNews_ACC_Network.csv')

In [ ]:
communities